# **Rendering tool for high quality molecular images and videos by POVARY**


In [111]:
from ase.build import molecule
from widget_periodictable import PTableWidget
import nglview as nv
from ipywidgets import HBox, VBox, Textarea, Button, Layout, ColorPicker, Checkbox, FileUpload, FloatSlider, Image
import numpy as np
from vapory import *
from numpy.linalg import norm
from copy import deepcopy
from ase.io import read, write
import matplotlib
import io
import json

In [112]:
with open("colors_jmol.json", "r") as fs:
    colors = json.load(fs)
    
with open("radius.json", "r") as fx:
    radius = json.load(fx)
    
fs.close()
fx.close()

In [113]:
aa = molecule("C60")
aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
aa.center()
aa.pbc=True
view = nv.show_ase(aa)
view.add_unitcell()
view.control.zoom(0.2)
view.add_ball_and_stick(aspectRatio=4)
view.camera='perspective'
view.background='white'

cp = ColorPicker(
    concise=False,
    description='Background color',
    value='white',
    disabled=False
)

def background_color_change(b):
    view.background = cp.value

cp.observe(background_color_change, names="value")

In [114]:
w = Textarea(
    value='C60',
    placeholder='Type your molecule',
    description='Molecule:',
    disabled=False,
    layout=Layout(width='28%', height='27px')
)

def _on_file_upload(change=None):
    global aa
    """When file upload button is pressed."""
    for fname, item in change['new'].items():
        frmt = fname.split('.')[-1]
        if frmt == 'xyz':
            aa = read(io.StringIO(item['content'].decode()), format='xyz')
        break
            

fupload = FileUpload(
    accept='.xyz',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

fupload.observe(_on_file_upload, names='value')

bond_factor = FloatSlider(value = 1.2, min = 0.5, max = 2.0, description="Factor of Bond")

In [115]:
PTable = PTableWidget(states=2, selected_colors = ['red','green'], selected_elements = {'C':0}, 
                      border_color='black', unselected_color = 'pink', width='18px')

In [116]:
def on_button_click(b):
    global view, aa
    aa = molecule(w.value)
    aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
    aa.center()
    aa.pbc=True
    for comp_id in view._ngl_component_ids:
        view.remove_component(comp_id)
    view.add_component(nv.ASEStructure(aa))
    view.clear()
    view.add_ball_and_stick(aspectRatio=4)
    view.add_unitcell()
    bcell.value = True
    view.control.zoom(0.2)
    PTable.selected_elements = {key: 0 for key in list(dict.fromkeys(aa.get_chemical_symbols()))}

def on_fileupload_click(b):
    global aa, view
    aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
    aa.center()
    aa.pbc=True
    for comp_id in view._ngl_component_ids:
        view.remove_component(comp_id)
    view.add_component(nv.ASEStructure(aa))
    view.clear()
    view.add_ball_and_stick(aspectRatio=4)
    view.add_unitcell()
    bcell.value = True
    view.control.zoom(0.2)
    PTable.selected_elements = {key: 0 for key in list(dict.fromkeys(aa.get_chemical_symbols()))}

def povray_render(b):
    br.disabled = True
    A = view._camera_orientation
    A = np.array(A)
    A=A.reshape(4,4)
    A=np.transpose(A)

    zfactor = norm(A[0, 0:3])
    A[0:3, 0:3] = A[0:3, 0:3]/zfactor
    
    bb = deepcopy(aa);

    for i in bb:
        a = np.array([i.x, i.y, i.z])
        a = a + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        i.x = -w[0]
        i.y = w[1]
        i.z = w[2]
        
    vertices = [];
    
    vx = np.array(bb.get_cell()[0]);
    vy = np.array(bb.get_cell()[1]);
    vz = np.array(bb.get_cell()[2]);

    vertices.append(np.array([0, 0, 0]));
    vertices.append(vx);
    vertices.append(vy);
    vertices.append(vz);
    
    vertices.append(vx+vy);
    vertices.append(vx+vz);
    vertices.append(vy+vz);
    vertices.append(vx+vy+vz);


    for n, i in enumerate(vertices):
        a = i + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        vertices[n] = np.array([-w[0], w[1], w[2]])
    

    camera = Camera('location', [0, 0, -zfactor/1.5], 'look_at', [0.0, 0.0, 0.0])

    light1 = LightSource([0, 0, -100.0], 'color',  [1.5, 1.5, 1.5])
    light2 = LightSource([-100.0, -100.0, -60], 'color',  [1.5, 1.5, 1.5])
    light3 = LightSource([0, -60.0, 0], 'color',  [1, 1, 1])

    #light = LightSource(-A[2][0:3], [1.3, 1.3, 1.3])
    wall = Plane([0, 0, 100], 20, Texture(Pigment('color', [1, 1, 1])))

    spheres = [];

    for i in bb:
        sphere = Sphere( [i.x, i.y, i.z], radius[i.symbol], 
                        Texture(Pigment( 'color', np.array(colors[i.symbol])/255)), 
                            Finish('phong', 0.9,'reflection', 0.05))
        spheres.append(sphere)


    bonds = [];
    for x, i in enumerate(bb):
        for j in bb[x+1:]:
            v1 = np.array([i.x, i.y, i.z])
            v2 = np.array([j.x, j.y, j.z])

            if i.symbol == 'H' and j.symbol == 'H':
                continue;
                
            if norm(v1-v2) < bond_factor.value*(radius[i.symbol] + radius[j.symbol]):
                bond = Cylinder(v1, v2, 0.2, Pigment('color', (np.array(colors[i.symbol])+np.array(colors[j.symbol]))/510),
                                Finish('phong', 0.8,'reflection', 0.05))
                bonds.append(bond)
                
    edges = [];
    for x, i in enumerate(vertices):
        for y, j in enumerate(vertices):
            if y > x:
                if norm(np.cross(i-j, vertices[1]-vertices[0])) < 0.001 or norm(np.cross(i-j, vertices[2]-vertices[0])) < 0.001 or norm(np.cross(i-j, vertices[3]-vertices[0])) < 0.001:
                    edge = Cylinder(i, j, 0.06, Texture(Pigment( 'color', [212/255.0,175/255.0,55/255.0])), 
                                    Finish('phong', 0.9,'reflection', 0.01))
                    edges.append(edge)

    objects = [light1] + spheres + bonds + [Background( "color", np.array(matplotlib.colors.to_rgb(view.background)))]
    
    if bcell.value:
        objects += edges
        
    scene = Scene( camera, objects= objects)
    #scene = Scene( camera, objects= [light1, wall] + spheres + bonds, included=["textures.inc"] )
    scene.render("nglview.png", width=3840, height=2160, antialiasing=0.000, quality=11, remove_temp=False)
    br.disabled = False

b = Button(description = 'Update')
br = Button(description = 'Render')
bu = Button(description = 'Upload File')

bcell = Checkbox(
    value=True,
    description='Show cellbox',
    disabled=False,
    indent=False
)

b.on_click(on_button_click)
bu.on_click(on_fileupload_click)
br.on_click(povray_render)



def checkbox_change(b):
    global view
    if bcell.value:
        view.add_unitcell()
    else:
        view.remove_unitcell()

bcell.observe(checkbox_change, names='value')

In [117]:
display(HBox([w, b]), HBox([fupload, bu]), VBox([cp, bcell]), bond_factor, br, VBox([view, PTable]))

FloatSlider(value=1.2, description='Factor of Bond', max=2.0, min=0.5)

Button(description='Render', style=ButtonStyle())